In [7]:
import pandas as pd
from omegaconf import OmegaConf
from lib.uncertainty import Simulate
from prettytable import PrettyTable

In [8]:
EXPERIMENTS = [
    '2026-01-26_20-22_fever_dense',
    '2026-01-26_20-23_fever_sparse',
    # '2026-01-26_20-23_fever_hybrid',
    '2026-01-26_23-23_fever_hybrid',
    '2026-01-26_20-25_fever_oracle',
    '2026-01-26_20-55_fever_random',
    '2026-01-26_20-23_fever_probabilistic',
    '2026-01-26_23-23_fever_probabilistic',
    '2026-01-26_21-44_compendium_dense',
    '2026-01-26_22-34_compendium_sparse',
    # '2026-01-26_22-42_compendium_hybrid',
    '2026-01-26_23-23_compendium_hybrid',
    '2026-01-26_21-56_compendium_oracle',
    '2026-01-26_22-05_compendium_random',
    '2026-01-26_22-52_compendium_probabilistic',
    '2026-01-26_23-39_compendium_probabilistic'
]

dataset = {
    'fever': 'Fever',
    'compendium': 'Compendium'
}

strategy = {
    'oracle': 'Oracle',
    'random': 'Random',
    'similar': 'Similar',
    'dense': 'Dense',
    'sparse': 'Sparse'
}

In [9]:
results_all = {}

for experiment in EXPERIMENTS:
    id = experiment.split('_')

    # cfg = OmegaConf.load(f'../results/{experiment}/config.yaml')
    results = pd.read_json(f'../results/{experiment}/results.json')
    results['correct_query'] = True

    results['date'] = id[0]
    results['time'] = id[1]
    results['dataset'] = id[2]

    results['retriever_strategy'] = id[3]

    results_all[experiment] = results

FileNotFoundError: File ../results/2026-01-26_23-39_compendium_probabilistic/results.json does not exist

In [ ]:
# cfg = OmegaConf.load(f'../results/{experiment}/config.yaml')
simulate = Simulate()

success_rates = {}
for experiment, results in results_all.items():
    success_rates[experiment] = simulate.compute_uncertainty(results)

In [ ]:
t = PrettyTable(field_names=['Dataset', 'Retriever Strategy', 'Query', 'Retreiver', 'Generator', '# Tests'])

for experiment, success_rate in success_rates.items():
    id = experiment.split('_')
    t.add_row([
        id[2],
        id[3],
        f"{success_rate['q']['mean']:.2%} ± {success_rate['q']['std']:.4%}",
        f"{success_rate['r']['mean']:.2%} ± {success_rate['r']['std']:.4%}",
        f"{success_rate['g']['mean']:.2%} ± {success_rate['g']['std']:.4%}",
        len(results_all[experiment])
    ])

t

Dataset,Retriever Strategy,Query,Retreiver,Generator,# Tests
fever,dense,99.99% ± 0.0099%,65.78% ± 0.4768%,78.83% ± 0.4078%,10000
fever,sparse,99.99% ± 0.0099%,51.07% ± 0.5032%,76.58% ± 0.4214%,10000
fever,hybrid,99.99% ± 0.0099%,51.07% ± 0.5032%,76.58% ± 0.4215%,10000
fever,oracle,99.99% ± 0.0099%,99.98% ± 0.0119%,91.33% ± 0.2816%,10000
fever,random,99.99% ± 0.0099%,3.08% ± 0.1741%,4.28% ± 0.1989%,10000
fever,probabilistic,99.99% ± 0.0099%,80.41% ± 0.3981%,73.86% ± 0.4439%,10000
compendium,dense,99.83% ± 0.1658%,82.42% ± 1.5547%,76.71% ± 1.7234%,598
compendium,sparse,99.83% ± 0.1658%,58.29% ± 2.0206%,55.94% ± 2.0271%,598
compendium,hybrid,99.83% ± 0.1658%,58.29% ± 2.0206%,55.60% ± 2.0299%,598
compendium,oracle,99.83% ± 0.1658%,99.58% ± 0.2590%,61.98% ± 1.9717%,598
